In [ ]:
import csv
import numpy as np
import pandas as pd

In [ ]:
x = []
y = []
with open ('/content/data.csv') as file:
  csv_reader = csv.reader(file)
  for row in csv_reader:
    if row[0]=='ID':
      continue
    datum = []
    datum.append(float(row[3]))
    datum.append(float(row[4]))
    if row[1]=='Female':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[2]))
    bmi = float(row[4])/(float(row[3])**2)
    datum.append(bmi)
    if row[5]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    if row[6]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[7]))
    datum.append(float(row[8]))
    if row[9]=='no':
      datum.append(0)
    elif row[9]=='Sometimes':
      datum.append(1)
    elif row[9]=='Frequently':
      datum.append(2)
    else:
      datum.append(3)
    #if row[10]=='yes':
      #datum.append(0)
    #else:
      #datum.append(1)
    datum.append(float(row[11]))
    if row[12]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[13]))
    datum.append(float(row[14]))
    if row[15]=='no':
      datum.append(0)
    elif row[15]=='Sometimes':
      datum.append(1)
    elif row[15]=='Frequently':
      datum.append(2)
    else:
      datum.append(3)
    if row[16]=='Walking':
      datum.append(0)
    elif row[16]=='Bike':
      datum.append(1)
    elif row[16]=='Motorbike':
      datum.append(2)
    elif row[16]=='Automobile':
      datum.append(3)
    else:
      datum.append(4)
    if row[17]=='Insufficient_Weight':
      y.append(0)
    elif row[17]=='Normal_Weight':
      y.append(1)
    elif row[17]=='Overweight_Level_I':
      y.append(2)
    elif row[17]=='Overweight_Level_II':
      y.append(3)
    elif row[17]=='Obesity_Type_I':
      y.append(4)
    elif row[17]=='Obesity_Type_II':
      y.append(5)
    else:
      y.append(6)
    x.append(datum)
print(x)
print(y)
x = np.array(x)
y = np.array(y)

In [ ]:
!pip install optuna

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import load_wine
import optuna

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [ ]:
model = XGBClassifier(use_label_encoder=False, 
                      eval_metric='mlogloss')
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }

    # Fit the model
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train, y_train)

    # Make predictions
    y_pred = optuna_model.predict(X_test)

    # Evaluate predictions
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

In [ ]:
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

In [ ]:
params = trial.params
model = XGBClassifier(**params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy after tuning: %.2f%%" % (accuracy * 100.0))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
!pip install shap

In [ ]:
import shap
import matplotlib.pyplot as plt
fig = plt.figure()
shap_values = shap.TreeExplainer(model).shap_values(X_test)
shap.summary_plot(shap_values, X_test, feature_names = ['Height', 'Weight', 'Gender', 'Age', 'BMI', 'family_history', 'FCHCF', 'FCV',
                                                        'NMM', 'CFBM', 'Smoke', 'CW', 'CCM', 'PAF', 'TUT', 'CA', 'Transportation']
                                     , class_names = ['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II',
                                                         'Obesity_Level_I', 'Obesity_Level_II', 'Obesity_Level_III'], show = False
                                     , class_inds = model.classes_)
plt.gcf().set_size_inches(20,10)
plt.show()

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold
model = XGBClassifier(**params)
skf = StratifiedKFold(n_splits=10)
scores = cross_val_score(model, x, y, cv=skf)

In [ ]:
print("Cross Validation Scores: ", scores)
print("Average CV: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))